In [1]:
import os

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import PIL.ImageOps
from sklearn.model_selection import train_test_split
from sklearn import datasets


import matplotlib.pyplot as plt
import seaborn as sns

import ray 
import cv2


import urllib
import requests

In [2]:
ray.shutdown()
ray.init()

2020-03-11 11:50:45,257	INFO resource_spec.py:216 -- Starting Ray with 5.66 GiB memory available for workers and up to 2.85 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '129.69.205.59',
 'redis_address': '129.69.205.59:20886',
 'object_store_address': '/tmp/ray/session_2020-03-11_11-50-45_245732_44912/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-11_11-50-45_245732_44912/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-03-11_11-50-45_245732_44912'}

In [7]:

files = ['fashionmnist','coil-100','paris_buildings', 'pets', 
         'oxford_buildings','dogs','cinic10', 'caltech',
         'flowers','paintings']


@ray.remote
def flatten_images(path, number=50):
    if not os.path.exists(path):
        return 1
    
    
    if os.path.exists(path.split('/')[0]+'/flat_img_{}.csv'.format(number)):
        print('Skipping....')
        return 
    images = []
    names = []
    for i,ff in enumerate(os.listdir(path)):
        if ff.startswith('.'):
            print('not an image: ' + ff)
            continue
        if i <2:
            print(ff.split('.')[0])
            
        img = cv2.imread(path + ff, cv2.IMREAD_UNCHANGED)
        if img is None:
            print('!!!! '+ ff)
            continue
        if len(img.shape) > 2 and img.shape[2] == 4:
            #convert the image from RGBA2RGB
            try:
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
            except:
                continue

        img = pd.DataFrame(np.array(img).flatten()).T
        #print(img)
        images.append(img)
        names.append(ff.split('.')[0])

    if len(images)>0:
        imgs = pd.concat(images, axis=0)
        imgs['filename'] = names
        imgs['labels'] = ['_'.join(n.split('_')[:-1]) for n in names]
        #print(imgs.head())
        imgs = imgs.dropna(axis=1)

        
        imgs.to_csv(path.split('/')[0]+'/flat_img_{}.csv'.format(number),index=False)
        print(imgs.head(2))
    print('----Finished: {}----'.format(path))

    return 0
    


for number in [50,200]:
    for f in files:
        if os.path.isdir(f) and not f.startswith('.') and not f.startswith('_') and not f.startswith('MNIST') and not f.startswith('textures'):
            print(f)
            if number == 50:
                path = f+'/thumbnails{}/'.format('')
            else:
                path = f+'/thumbnails{}/'.format(number)
            if not os.path.exists(path):
                print('x')
                continue
            else:
                flatten_images.remote(path, number)
        
        

dogs
fashionmnist
pets
oxford_buildings
flowers
paintings
cinic10
caltech
paris_buildings
coil-100
dogs
fashionmnist
pets
oxford_buildings
flowers
paintings
cinic10
caltech
paris_buildings
coil-100
(pid=43891) oxford_buildings/thumbnails/
(pid=43889) fashionmnist/thumbnails/
(pid=43891) ashmolean_000007
(pid=43891) radcliffe_camera_000015
(pid=43888) cinic10/thumbnails/
(pid=43885) caltech/thumbnails/
(pid=43887) paintings/thumbnails/
(pid=43887)  boat_94
(pid=43887)  boat_57
(pid=43884) flowers/thumbnails/
(pid=43886) dogs/thumbnails/
(pid=43890) pets/thumbnails/
(pid=43890) american_pit_bull_terrier_172
(pid=43890) boxer_190
(pid=43889) 5_2091
(pid=43884) Tulip_574
(pid=43889) 1_5509
(pid=43885) emu_0025
(pid=43885) minaret_0060
(pid=43884) Snowdrop_102
(pid=43891) not an image: .DS_Store
(pid=43886) n02089078-black-and-tan_coonhound_3419
(pid=43886) vizsla_11888
(pid=43887) 153
(pid=43888) truck_7234
(pid=43888) truck_17843
(pid=43891) 567
(pid=43884) 1353
(pid=43890) 1812
(pid=4388

(pid=43888) 7200
(pid=43886)      0  ...                             labels
(pid=43886) 0  186  ...  n02089078-black-and-tan_coonhound
(pid=43886) 0   64  ...                             vizsla
(pid=43886) 
(pid=43886) [2 rows x 7502 columns]
(pid=43886) ----Finished: dogs/thumbnails/----
(pid=43889)     0    1  ...                       filename                     labels
(pid=43889) 0  43   44  ...  american_pit_bull_terrier_172  american_pit_bull_terrier
(pid=43889) 0  91  121  ...                      boxer_190                      boxer
(pid=43889) 
(pid=43889) [2 rows x 12314 columns]
(pid=43889) ----Finished: pets/thumbnails200/----
(pid=43885)     0    1    2   3    4    5  ...  7356  7357  7358  7359      filename   labels
(pid=43885) 0  93  142  204  81  132  194  ...    24    40    33    63      emu_0025      emu
(pid=43885) 0   0    0    0   0    0    0  ...     0     0     1     1  minaret_0060  minaret
(pid=43885) 
(pid=43885) [2 rows x 7362 columns]
(pid=43885) ----Finis

In [6]:
files = ['paintings']
classes = [15,10,10,10,10,10,10,10]
number = 50


def reformat(f, number, c=10):
    print('Started: {}'.format(f))
    path = f+'/flat_img_{}.csv'.format(number)
    
#     if os.path.exists(f+'/flat_img_{}_{}classes.csv'.format(number, c)):
#         print('already done')
#         return
    
    try:
        df = pd.read_csv(path)
    except e:
        print('didnt work')
        print(e)
        return

    print('Before: {}'.format(df.shape))
    df_agg = df.groupby(['labels']).count()['filename'].reset_index().sort_values(['filename'], ascending=False)
    ll = df_agg.reset_index().loc[:c, 'labels']
    print('{} \n ---- \n {}'.format(f,df_agg.head(c)))
    df = df.loc[df['labels'].isin(ll)]
    df.to_csv(f+'/flat_img_{}_{}classes.csv'.format(number, c), index=False)
    print('After: {}'.format(df.shape))
    print('Finished: {}'.format(f))
    return 1


for number in [50]:
    for f in files:
        reformat(f, number)
            

Started: paintings
Before: (153, 7502)
paintings 
 ---- 
           labels  filename
19         horse        48
8           boat        25
17           dog        18
2           bird        10
13           cow        10
21         train         8
10         chair         7
16   diningtable         5
20         sheep         4
18     dog horse         3
After: (141, 7502)
Finished: paintings
